In [ ]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
import random
from datetime import datetime
import matplotlib.pyplot as plt

def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

class FruitCNN(nn.Module):
    def __init__(self):
        super(FruitCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(128 * 12 * 12, 256), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(256, 4)
        )

    def forward(self, x):
        return self.model(x)

def run_grid_search(train_path, test_path, batch_size=32, epochs=15, seed=42):
    set_seed(seed)

    transform_train = transforms.Compose([
        transforms.Resize((100, 100)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3)
    ])
    transform_test = transforms.Compose([
        transforms.Resize((100, 100)),
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3)
    ])

    train_data = ImageFolder(train_path, transform=transform_train)
    test_data = ImageFolder(test_path, transform=transform_test)

    train_size = int(0.7 * len(train_data))
    val_size = len(train_data) - train_size
    train_dataset, val_dataset = random_split(train_data, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Classes:", train_data.classes)
    print("Using device:", device)

    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    os.makedirs("results", exist_ok=True)
    csv_path = f"results/results_log_{timestamp}.csv"
    report_path = f"results/classification_reports_{timestamp}.txt"

    configurations = [
        ('SGD', 0.01, 0.5),
        ('Adam', 0.01, 0.5),
        ('AdamW', 0.005, 0.9)
    ]

    all_results = []
    with open(report_path, "w") as report_file:
        for opt_name, lr, lrd in configurations:
            print(f"\n=== Training with {opt_name}, lr={lr}, lr_decay={lrd} ===")
            model = FruitCNN().to(device)
            criterion = nn.CrossEntropyLoss()

            if opt_name == 'SGD':
                optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
            elif opt_name == 'Adam':
                optimizer = optim.Adam(model.parameters(), lr=lr)
            elif opt_name == 'AdamW':
                optimizer = optim.AdamW(model.parameters(), lr=lr)
            else:
                raise ValueError(f"Unknown optimizer: {opt_name}")

            scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=lrd)

            train_acc, val_acc, train_loss, val_loss = [], [], [], []

            for epoch in range(epochs):
                model.train()
                correct, total, running_loss = 0, 0, 0.0
                for images, labels in train_loader:
                    images, labels = images.to(device), labels.to(device)
                    optimizer.zero_grad()
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    running_loss += loss.item()
                    _, predicted = torch.max(outputs, 1)
                    correct += (predicted == labels).sum().item()
                    total += labels.size(0)
                train_acc.append(correct / total)
                train_loss.append(running_loss)

                model.eval()
                val_correct, val_total, val_running_loss = 0, 0, 0.0
                with torch.no_grad():
                    for val_images, val_labels in val_loader:
                        val_images, val_labels = val_images.to(device), val_labels.to(device)
                        val_outputs = model(val_images)
                        val_loss_batch = criterion(val_outputs, val_labels)
                        val_running_loss += val_loss_batch.item()
                        _, val_preds = torch.max(val_outputs, 1)
                        val_total += val_labels.size(0)
                        val_correct += (val_preds == val_labels).sum().item()
                val_acc.append(val_correct / val_total)
                val_loss.append(val_running_loss)
                scheduler.step()

                print(f"Epoch {epoch+1}, Train Acc: {train_acc[-1]:.4f}, Val Acc: {val_acc[-1]:.4f}")

            model.eval()
            y_true, y_pred = [], []
            with torch.no_grad():
                for images, labels in test_loader:
                    images = images.to(device)
                    outputs = model(images)
                    _, preds = torch.max(outputs, 1)
                    y_pred.extend(preds.cpu().numpy())
                    y_true.extend(labels.numpy())

            report = classification_report(y_true, y_pred, target_names=train_data.classes)
            print("\nClassification Report:")
            print(report)

            report_file.write(
                f"=== Optimizer: {opt_name}, LR: {lr}, Decay: {lrd} ===\n{report}\n\n"
            )

            all_results.append({
                'optimizer': opt_name,
                'lr': lr,
                'lr_decay': lrd,
                'final_train_acc': train_acc[-1],
                'final_val_acc': val_acc[-1],
                'final_train_loss': train_loss[-1],
                'final_val_loss': val_loss[-1]
            })

            # Plot Accuracy
            plt.figure()
            plt.plot(train_acc, label='Train Acc')
            plt.plot(val_acc, label='Val Acc')
            plt.title(f'{opt_name} lr={lr}, decay={lrd} - Accuracy')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.legend()
            plt.grid(True)
            plt.tight_layout()
            plt.show()

            # Plot Loss
            plt.figure()
            plt.plot(train_loss, label='Train Loss')
            plt.plot(val_loss, label='Val Loss')
            plt.title(f'{opt_name} lr={lr}, decay={lrd} - Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.legend()
            plt.grid(True)
            plt.tight_layout()
            plt.show()

            del model
            torch.cuda.empty_cache()

    pd.DataFrame(all_results).to_csv(csv_path, index=False)
    print(f"\nSaved training results to {csv_path}")
    print(f"Saved classification reports to {report_path}")

# Run
run_grid_search(
    train_path="../train",
    test_path="../test",
    batch_size=32,
    epochs=15,
    seed=42
)
